# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1a14a04430>
├── 'a' --> tensor([[-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877]])
└── 'x' --> <FastTreeValue 0x7f1a14a04370>
    └── 'c' --> tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                        [ 1.1404,  0.2478, -0.8150,  0.3418],
                        [ 0.1654,  0.1064,  1.0439,  3.0828]])

In [4]:
t.a

tensor([[-0.0037, -1.8884,  0.3552],
        [ 0.6655,  1.4680,  0.3877]])

In [5]:
%timeit t.a

81 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1a14a04430>
├── 'a' --> tensor([[ 2.0022,  0.5882,  0.8911],
│                   [-0.0700,  0.3933, -1.0834]])
└── 'x' --> <FastTreeValue 0x7f1a14a04370>
    └── 'c' --> tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                        [ 1.1404,  0.2478, -0.8150,  0.3418],
                        [ 0.1654,  0.1064,  1.0439,  3.0828]])

In [7]:
%timeit t.a = new_value

87.9 ns ± 3.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877]]),
    x: Batch(
           c: tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                      [ 1.1404,  0.2478, -0.8150,  0.3418],
                      [ 0.1654,  0.1064,  1.0439,  3.0828]]),
       ),
)

In [10]:
b.a

tensor([[-0.0037, -1.8884,  0.3552],
        [ 0.6655,  1.4680,  0.3877]])

In [11]:
%timeit b.a

70.9 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1461, -0.3928,  0.5166],
               [ 0.9569,  0.6588, -0.7642]]),
    x: Batch(
           c: tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                      [ 1.1404,  0.2478, -0.8150,  0.3418],
                      [ 0.1654,  0.1064,  1.0439,  3.0828]]),
       ),
)

In [13]:
%timeit b.a = new_value

629 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 19.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.5 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

274 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

259 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1a14a04640>
├── 'a' --> tensor([[[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]],
│           
│                   [[-0.0037, -1.8884,  0.3552],
│                    [ 0.6655,  1.4680,  0.3877]]])
└── 'x' --> <FastTreeValue 0x7f194fdbb760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.7 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f194fdbbfa0>
├── 'a' --> tensor([[-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877],
│                   [-0.0037, -1.8884,  0.3552],
│                   [ 0.6655,  1.4680,  0.3877]])
└── 'x' --> <FastTreeValue 0x7f194fdbbac0>
    └── 'c' --> tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                        [ 1.1404,  0.2478, -0.8150,  0.3418],
                 

In [23]:
%timeit t_cat(trees)

43.9 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

99.6 µs ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]],
       
               [[-0.0037, -1.8884,  0.3552],
                [ 0.6655,  1.4680,  0.3877]]]),
    x: Batch(
           c: tensor([[[ 0.5997,  0.9202,  0.4062, -0.2686],
                       [ 1.1404,  0.2478, -0.8150,  0.3418],
                       [ 0.1654,  0.1064,  1.0439,  3.0828]],
         

In [26]:
%timeit Batch.stack(batches)

111 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877],
               [-0.0037, -1.8884,  0.3552],
               [ 0.6655,  1.4680,  0.3877]]),
    x: Batch(
           c: tensor([[ 0.5997,  0.9202,  0.4062, -0.2686],
                      [ 1.1404,  0.2478, -0.8150,  0.3418],
                      [ 0.1654,  0.1064,  1.0439,  3.0828],
                      [ 0.5997,  0.9202,  0.4062, -0.2686],
                      [ 1.1404,  

In [28]:
%timeit Batch.cat(batches)

255 µs ± 7.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

625 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
